# LLM-for-Metadata-Harvesting

This notebook contains the experimental results from [P6: Groot zeegras (2023)](https://datahuiswadden.openearth.nl/geonetwork/srv/api/records/TF1TbsTxTqykP5rv6MXJEg).  
The results can be found under the last code block. Note that not all code is directly relevant to this experiment; some parts are retained for future development and elaboration.


In [1]:
from cheatsheet import CHEATSHEETS
from prompt import PROMPTS
from grobidmonkey import reader
from webutils import readWebContent, downloadAndParseXML

monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

dataPortalURL = [
    "https://developers.google.com/earth-engine/datasets/catalog/NASA_HLS_HLSS30_v002",
    "https://lpdaac.usgs.gov/products/mod09a1v061/",
    "https://stac.ecodatacube.eu/veg_quercus.robur_anv.eml/collection.json?.language=en",
    "https://stac.ecodatacube.eu/ndvi_glad.landsat.ard2.seasconv/collection.json?.language=en",
    "https://zenodo.org/records/8319440",
    "https://lifesciences.datastations.nl/dataset.xhtml?persistentId=doi:10.17026/dans-2bd-kskz",
    "https://www.gbif.org/dataset/4fa7b334-ce0d-4e88-aaae-2e0c138d049e",
    "https://www.gbif.org/dataset/74196cd9-7ebc-4b20-bc27-3c2d22e31ed7",
    "https://www.gbif.org/dataset/bc0acb9a-131f-4085-93ae-a46e08564ac5",
    "https://zenodo.org/records/11440456",
    "https://stac.ecodatacube.eu/blue_glad.landsat.ard2.seasconv.m.yearly/collection.json",
    "https://datahuiswadden.openearth.nl/geonetwork/srv/eng/catalog.search#/metadata/L-mHomzGRuKAHGMkUPjY9g",
    "https://datahuiswadden.openearth.nl/geonetwork/srv/eng/catalog.search#/metadata/0fe7e64b-50b3-4cee-b64a-02659fc2b6c7",
    "https://stac.ecodatacube.eu/green_glad.landsat.ard2.seasconv.m.yearly/collection.json",
    "https://datahuiswadden.openearth.nl/geonetwork/srv/api/records/A0h06_NlSEuNlium5OO3FA",
]

# Get the web content
url = dataPortalURL[0]


# soup = readWebContent(url)
# if soup is None:
#     raise ValueError("Failed to retrieve web content")

# # Extract text from the webpage - adjust the selector based on the webpage structure
# # This is a basic example - you might need to modify based on the specific webpage
# text = soup.get_text(separator='\n', strip=True)

# text_xml, _ = downloadAndParseXML("https://datahuiswadden.openearth.nl/geonetwork/srv/api/records/A0h06_NlSEuNlium5OO3FA/formatters/xml")
# text += "\n" + text_xml
# full_text = text
import nest_asyncio
import asyncio
from webutils import extract_full_page_text

# Apply nest_asyncio to allow asyncio.run() in Jupyter
nest_asyncio.apply()

# Run the async function
full_text = await extract_full_page_text(url)

# Optionally display or save it
print(full_text)  # Print the first 1000 characters

Skip to main content

developers.google.com uses cookies from Google to deliver and enhance the quality of its services and to analyze traffic. Learn more.

OK, got it
Earth Engine Data Catalog
/
Sign in
Home
Categories
All datasets
All tags
Landsat
MODIS
Sentinel
Publisher
Community
API Docs
HLSS30: HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m 
bookmark_border
Dataset Availability
2015-11-28T00:00:00Z–2025-05-06T23:38:31Z
Dataset Provider
NASA LP DAAC
Earth Engine Snippet
ee.ImageCollection("NASA/HLS/HLSS30/v002") open_in_new
Tags
landsat nasa satellite-imagery sentinel usgs
Description
Bands
Image Properties
Terms of Use
Citations
DOIs

The Harmonized Landsat Sentinel-2 (HLS) project provides consistent surface reflectance data from the Operational Land Imager (OLI) aboard the joint NASA/USGS Landsat 8 satellite and the Multi-Spectral Instrument (MSI) aboard Europe's Copernicus Sentinel-2A satellites. The combined measurement enables global observatio

In [2]:
from tqdm import tqdm
from hearvester_operations import extract_entities
initial_url_map = {}
clean_url_map = {}

index = 0

for url in tqdm(dataPortalURL):
    index += 1
    # Apply nest_asyncio to allow asyncio.run() in Jupyter
    nest_asyncio.apply()

    # Run the async function
    if url.startswith("https://lpdaac.usgs.gov"):
        soup = readWebContent(url)
        if soup is None:
            raise ValueError("Failed to retrieve web content")

        # Extract text from the webpage - adjust the selector based on the webpage structure
        # This is a basic example - you might need to modify based on the specific webpage
        full_text = soup.get_text(separator='\n', strip=True)
    else:
        full_text = await extract_full_page_text(url)

    # Optionally display or save it
    special_interest = CHEATSHEETS.get("special_interests", "Focus on metadata fields and their relationships")
    initial_nodes, clean_nodes = extract_entities(
        text=full_text, 
        entity_types=PROMPTS["DEFAULT_ENTITY_TYPES"], 
        special_interest=special_interest
    )

    initial_entity_type_map = {}

    for entity_group in initial_nodes.values():
        for item in entity_group:
            entity_name = item.get('entity_name')
            entity_type = item.get('entity_type')
            entity_description = item.get('description')

            # Initialize the list for this entity_type if not already present
            if entity_name not in initial_entity_type_map:
                initial_entity_type_map[entity_type] = []

            # Append the (entity_name, description) pair
            initial_entity_type_map[entity_type].append(entity_name + '; ' + entity_description)
    initial_url_map[url] = initial_entity_type_map

    # Create a dictionary to store entity_type: [(entity_name, description), ...]
    clean_entity_type_map = {}

    for entity_group in clean_nodes.values():
        for item in entity_group:
            entity_name = item.get('entity_name')
            entity_value = item.get('entity_value')

            # Initialize the list for this entity_type if not already present
            if entity_name not in clean_entity_type_map:
                clean_entity_type_map[entity_name] = []

            # Append the (entity_name, description) pair
            clean_entity_type_map[entity_name].append(entity_value)
    clean_url_map[url] = clean_entity_type_map

100%|██████████| 15/15 [19:08<00:00, 76.54s/it]


In [6]:
import yaml
import os

from datetime import datetime
now = datetime.now()
date_str = now.strftime("%Y-%m-%d")

output_file_path = "outputs/" + date_str + "/" + "clean_entity_type_map.yaml"
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Sort the inner dictionary keys for each dataset URL
sorted_data = {
    url: dict(sorted(fields.items()))
    for url, fields in clean_url_map.items()
}

# Save to YAML
with open(output_file_path, "w") as file:
    yaml.dump(sorted_data, file, sort_keys=False, allow_unicode=True)

In [ ]:
import yaml
import os

from datetime import datetime
now = datetime.now()
date_str = now.strftime("%Y-%m-%d")

output_file_path = "outputs/" + date_str + "/" + "initial_entity_type_map.yaml"
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Sort the inner dictionary keys for each dataset URL
sorted_data = {
    url: dict(sorted(fields.items()))
    for url, fields in initial_url_map.items()
}

# Save to YAML
with open(output_file_path, "w") as file:
    yaml.dump(sorted_data, file, sort_keys=False, allow_unicode=True)

In [ ]:
def parse_metadata_block(text: str) -> tuple[str, dict]:
    """
    Parse a metadata block and return the source URL and a dictionary of metadata fields.
    
    Args:
        text (str): Text block containing metadata information
        
    Returns:
        tuple: (source_url, metadata_dict) where metadata_dict contains field types as keys 
        and lists of (value, description) tuples as values
    """
    lines = text.split('\n')
    metadata = {}
    source_url = ""
    current_field = None
    
    for line in lines:
        line = line.strip()
        
        # Skip empty lines and separator lines
        if not line or line.startswith('==='):
            continue
            
        # Extract source URL
        if line.startswith('Source URL:'):
            source_url = line.replace('Source URL:', '').strip()
            continue
            
        # Check for field type
        if line and not line.startswith('-'):
            if line.endswith(':'):
                current_field = line[:-1]  # Remove trailing colon
                metadata[current_field] = []
            continue
            
        # Process metadata entries
        if line.startswith('- (') and current_field:
            # Extract content between parentheses
            content = line[5:-1]  # Remove "  - (" and ")"
            if ',' in content:
                # Split into value and description
                value, desc = content.split(',', 1)
                value = value.strip()
                desc = desc.strip()
                metadata[current_field].append((value, desc))
    
    return source_url, metadata

def read_metadata_file(file_path: str) -> list[tuple[str, dict]]:
    """
    Read the metadata file and parse all blocks.
    
    Args:
        file_path (str): Path to the metadata file
        
    Returns:
        list: List of (source_url, metadata_dict) tuples for each block
    """
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Split content into blocks using the separator
    sep_blocks = content.split('\n==================================================\n')
    blocks = []

    print(len(sep_blocks))

    if len(sep_blocks)%2:
        for i in range(1, len(sep_blocks)-1, 2):
            block = sep_blocks[i] + "\n" + sep_blocks[i+1]
            blocks.append(block)
    else:
        raise ValueError("The file content does not have the expected format.")

    
    # Parse each non-empty block
    results = []
    for block in blocks:
        if block.strip():
            url, metadata = parse_metadata_block(block)
            if url:  # Only add blocks with a valid source URL
                results.append((url, metadata))
    
    return results

# Example usage:
file_path = "outputs/2025-05-20entity_type_map.txt"
metadata_blocks = read_metadata_file(file_path)

# Print example of first block
if metadata_blocks:
    url, metadata = metadata_blocks[0]
    print(f"Source URL: {url}")
    print("\nExample fields:")
    for field, values in list(metadata.items())[:3]:  # Show first 3 fields
        print(f"\n{field}:")
        for value, desc in values:
            print(f"  - {value}: {desc}")

In [ ]:
metadata_blocks


In [6]:
# create_dill_pickle.py
import dill

class MyModel:
    def get_team(self):
        return "Awesome Hackers"

model = MyModel()

# Save with dill
with open("my_model.pkl", "wb") as f:
    dill.dump(model, f, protocol=4)

print("Pickle file created using dill: my_model.pkl")

Pickle file created using dill: my_model.pkl


In [4]:
pip install dill

Note: you may need to restart the kernel to use updated packages.
